In [2]:
from scipy.sparse import lil_matrix
import pickle
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [5]:
def load_stat_subclusters(stat_subclusters_file_path):
    stat_subclusters = list()
    with open(stat_subclusters_file_path, 'r') as infile:
        infile.readline() # skip header
        for line in infile:
            subcluster = set(line.split("\t")[5].split(","))
            stat_subclusters.append(subcluster)
    return stat_subclusters


stat_subclusters_file_path = "../data/PRESTO-STAT_subclusters.txt"
stat_subclusters = load_stat_subclusters(stat_subclusters_file_path)

stat_subclusters[0]

{'ACPS_c6',
 'AMP-binding_c30;AMP-binding_C_c39',
 'Chorismate_bind_c14',
 'Isochorismatase_c20;PP-binding_c39'}

In [ ]:
def jaccard_similarity(list1, list2):
    intersection = len(set(list1) & set(list2))
    union = len(set(list1) | set(list2))
    return intersection / union

# Build a similarity matrix
n = len(stat_subclusters)
similarity_matrix = lil_matrix((n, n))

for i in range(n):
    for j in range(n):
        similarity_matrix[i, j] = jaccard_similarity(stat_subclusters[i], stat_subclusters[j])

with open('similarity_matrix.pkl', 'wb') as f:
    pickle.dump(similarity_matrix, f)

In [ ]:
with open("../results/similarity_matrix.pkl", "rb") as f:
    similarity_matrix = pickle.load(f)

# Convert to distance matrix for clustering
distance_matrix = 1 - similarity_matrix.toarray()

del similarity_matrix

with open("../results/distance_matrix.pkl", "wb") as f:
    pickle.dump(distance_matrix, f)

In [ ]:
with open("../results/distance_matrix.pkl", "rb") as f:
    distance_matrix = pickle.load(f)

# Apply clustering
cluster_model = AgglomerativeClustering(metric='precomputed', linkage='average')
labels = cluster_model.fit_predict(distance_matrix)